In [1]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pymysql

In [2]:
def craw_html(my_dict):
    html = driver.page_source
    bs = BeautifulSoup(html,'html.parser')
    trs = bs.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
    for tr in trs:
        href = tr.select('a')[0]['href']
        play_id = href.split('=')[1]
        play_name = tr.select('a')[0].text.replace('*','')
        g_num = tr.find('td',{'data-id':'GAME_CN'}).text
        my_dict[play_id] = play_name.strip()
    return my_dict

In [3]:
url = 'https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicTotal.aspx'
driver = webdriver.Chrome('E:/BigData/BigData/driver/chromedriver.exe')
# driver = webdriver.Chrome('E:/chromedriver_win32/chromedriver.exe')
driver.get(url)
time.sleep(1)

my_dict = {}
start_time = time.time()
select = Select(driver.find_element_by_id('cphContents_cphContents_cphContents_ddlSeason_ddlSeason'))
select.select_by_value('2020')
time.sleep(1)

driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > thead > tr > th:nth-child(5) > a').click()
time.sleep(1)

page_num=1
count=0

print('start!!!')

while True:
    try:
        craw_html(my_dict)
        time.sleep(1)
        page_num += 1
        if page_num > 5:
            driver.find_element_by_css_selector('#cphContents_cphContents_cphContents_ucPager_btnNext').click()
            page_num = 1
        else:
            page_tag = '#cphContents_cphContents_cphContents_ucPager_btnNo{}'.format(page_num)
            driver.find_element_by_css_selector(page_tag).click()   
        time.sleep(1)
        count+=1
        if count >= 50:
            break
    except:
        print('페이지 완료')
        break
        
end_time = time.time()
total_time = end_time - start_time
print('='*60)
print(total_time)

start!!!
페이지 완료
18.734086990356445


In [4]:
list(my_dict.keys())
df = pd.DataFrame({'play_id':list(my_dict.keys()),'play_name':list(my_dict.values())})

In [11]:
df.head()

,play_id,play_name
0,65060,주권
1,64995,배재환
2,67737,김진영
3,63894,김정빈
4,64893,박민호


In [12]:
df.to_csv('./kbo_pitcher.csv',encoding='utf-8-sig')